In [216]:
import json 
import time
import requests
import urllib, urllib2
from lxml import html
from lxml import etree
import locale
import random
import pymongo
from selenium import webdriver  
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [208]:
#Connect to mongo
conn = pymongo.MongoClient()
db = conn.faces
collection = db.urlClubPhotos

In [212]:
#Set toScroll = False for testing & fbPageName is name of club's fb page
listOfUrls = []
listOfImageUrls = []
def getPicUrls(fbPageName, toScroll):
    pg = 0
    driver = webdriver.Chrome()
    driver.get('https://www.facebook.com/' + fbPageName + '/photos_stream?')

    url = 'https://www.facebook.com/' + fbPageName
    r = requests.get(url)
    facebookID = r.text.split('"pageID":"')[1].split('"')[0]
    svid = '5|' + r.text.encode('utf-8').split('actorid":"')[1].split('"')[0]   
    #Get thumbnails of images    
    scroll = True
    while scroll and pg <= 2:
        pg += 1
        print(pg)
        a = len(listOfUrls)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        for i in driver.find_elements_by_xpath("//*[@class='uiMediaThumb _6i9 uiMediaThumbMedium']"):
            tempDict = {}
            tempDict['thumbUrl'] = i.get_attribute("href")
            tempDict['_id'] = tempDict['thumbUrl']
            tempDict['state'] = 0
            tempDict['svid'] = svid
            listOfUrls.append(tempDict['thumbUrl'])
            r = requests.get(tempDict['thumbUrl'])
            tree = html.fromstring(r.text)
            for htmlObject in tree.xpath("//img[@class='fbPhotoImage img']"): 
                tempDict['urlOfImage'] = htmlObject.attrib['src']
                listOfImageUrls.append(tempDict['urlOfImage'])
            for htmlObject2 in tree.xpath("//abbr"):  
                tempDict['albumTitle'] = htmlObject2.attrib['title']
                tempDict['uploadTime'] = htmlObject2.attrib['data-utime']
                tempDict['fbPageName'] = fbPageName
            try:    
                collection.insert(tempDict)
            except:
                None
                
        scroll = toScroll

        b = len(listOfUrls)

        
        if a == b:
            scroll = False



In [29]:
fbPageName='opiumbarcelona'

In [214]:
getPicUrls(fbPageName, True)


1
2
3


In [167]:
r=collection.find()
for i in r:
    print(i["thumbUrl"])
    
collection.count()    


https://www.facebook.com/opiumbarcelona/photos/pb.143141875746969.-2207520000.1443793167./975543955840086/?type=3&src=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xfa1%2Ft31.0-8%2F12015109_975543955840086_6225592736943142083_o.jpg&smallsrc=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xfa1%2Ft31.0-8%2Fs960x960%2F12015109_975543955840086_6225592736943142083_o.jpg&size=973%2C1460&fbid=975543955840086
https://www.facebook.com/opiumbarcelona/photos/pb.143141875746969.-2207520000.1443793167./975543935840088/?type=3&src=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xat1%2Ft31.0-8%2F12068973_975543935840088_1512905081278260515_o.jpg&smallsrc=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xat1%2Fv%2Ft1.0-9%2F12088275_975543935840088_1512905081278260515_n.jpg%3Foh%3Dccadca5949463e72d36938539c928d38%26oe%3D5691E9A1&size=1460%2C973&fbid=975543935840088
https://www.facebook.com/opiumbarcelona/photos/pb.143141875746969.-2207520000.1443793167./975543865840095/?type=3&src=

294

In [215]:
x = collection.distinct("thumbUrl")
y = [i for i in collection.find({}, {"thumbUrl":1, "_id":0})]

pipe = [{'$group': { '_id': { 'thumbUrl': '$thumbUrl' }, 'urlCount': { '$sum': 1 }}}]
collection.aggregate(pipeline=pipe)

{u'ok': 1.0,
 u'result': [{u'_id': {u'thumbUrl': u'https://www.facebook.com/opiumbarcelona/photos/pb.143141875746969.-2207520000.1443802211./974243625970119/?type=3&src=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xat1%2Ft31.0-8%2F12017575_974243625970119_1995225825056414799_o.jpg&smallsrc=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xaf1%2Fv%2Ft1.0-9%2F12019963_974243625970119_1995225825056414799_n.jpg%3Foh%3D58c27dcbe730f3c0f342432c5bfe3b8e%26oe%3D5689C02C&size=2048%2C1365&fbid=974243625970119'},
   u'urlCount': 1},
  {u'_id': {u'thumbUrl': u'https://www.facebook.com/opiumbarcelona/photos/pb.143141875746969.-2207520000.1443802211./974243679303447/?type=3&src=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xtf1%2Ft31.0-8%2F12031364_974243679303447_6157179662715520818_o.jpg&smallsrc=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xfp1%2Fv%2Ft1.0-9%2F12049217_974243679303447_6157179662715520818_n.jpg%3Foh%3Dab09f01d7788fd6bb8471e93bebf99af%26oe%3D568647E9&size=

In [217]:
conn = pymongo.MongoClient()
db = conn.faces
collection1 = db.facesMeta


In [106]:
x=collection.find()
for i in x:
    print(i)
#cursor1= collection.find({state : {"$eq" : 0}})

{u'urlOfImage': u'https://scontent-mad1-1.xx.fbcdn.net/hphotos-xaf1/v/t1.0-9/s720x720/12042845_975095622551586_7459912233832932325_n.jpg?oh=46c36d9469859912aec49fc5fcc41e2d&oe=5689DCA1', u'uploadTime': u'1443647315', u'thumbUrl': u'https://www.facebook.com/opiumbarcelona/photos/pb.143141875746969.-2207520000.1443705403./975095622551586/?type=3&src=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xat1%2Ft31.0-8%2F12080353_975095622551586_7459912233832932325_o.jpg&smallsrc=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xaf1%2Fv%2Ft1.0-9%2F12042845_975095622551586_7459912233832932325_n.jpg%3Foh%3D2685fcc20247d4a0aa155dbbc4ca8485%26oe%3D56988FC3&size=2048%2C1365&fbid=975095622551586', u'albumTitle': u'Mi\xe9rcoles, 30 de septiembre de 2015 a las 14:08', u'fbPageName': u'opiumbarcelona', u'svid': u'5|143141875746969', u'state': 0, u'_id': ObjectId('560d3249aea396329d27bf02')}
{u'urlOfImage': u'https://scontent-mad1-1.xx.fbcdn.net/hphotos-xap1/v/t1.0-9/s720x720/12063698_97509561588

In [162]:
from __future__ import division
from facepp import API, File
import json
from pandas.io.json import json_normalize
import pandas as pd
import time
import cv2
from pandas import compat
from findtools.find_files import (find_files, Match)
import codecs
from pandas import compat
import ast, json
from pandas import DataFrame
import yaml
import numpy as np

import simplejson
import tmpFB
import time
import codecs
import ast

api = API('b40d3ea28aad1f2c2aa939dec674abff', '_gYS77zBAL0U60gtv_qebqwRmRWJz4c1')

def get_face_meta_url(api):
    counter = 0
    facesMeta = pd.DataFrame()
    facesMetaList = []
    noFaces = []
    cursor1 = collection.find({'state': 0})
    
    for record in cursor1:
        print('check')
        counter += 1  
        try:
            result = api.detection.detect(url = record['urlOfImage'],  attribute='pose,age,gender,race,smiling')
            r_dump = json.dumps(result)
            result2 = yaml.safe_load(r_dump)

            if counter % 100 == 0:
                print('still going')

            if len(result['face']) > 0:
                collection.update({'urlOfImage': record['urlOfImage']}, {'$set': {'state': 1}})
                z = collection.find({},  {'state':1, '_id':0})
                for face in result['face']:
                    tempDict = face
                    tempDict['url'] = record['urlOfImage']
                    tempDict['img_width'] = result['img_width']
                    tempDict['img_height'] = result['img_height']
                    tempDict['svid'] = record['svid']
                    tempDict['fbPageName'] = record['fbPageName']
                    collection1.insert(tempDict)               
                    facesMetaList.append(tempDict)

            if len(result['face']) == 0:
                noFaces.append(record['urlOfImage'])
                collection.update({'urlOfImage': record['urlOfImage']}, {'$set': {'state': 1}})
        except Exception as x:
            print('e')
            if x.code == 432:
                failedLinks.append(imgpath)
            elif x.code == 502:
                time.sleep(60)
                print(idx)
            else:
                print(imgpath)
            failedLinks.append(imgpath)    

http://api.us.faceplusplus.com/


In [163]:
get_face_meta_url(api)

check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check


In [161]:
collection.update({'state':  {'$gt': -1}}, {'$set': {'state': 0}}, multi=True)
x=collection.find()
for i in x:
    print(i)

{u'urlOfImage': u'https://scontent-mad1-1.xx.fbcdn.net/hphotos-xaf1/v/t1.0-9/s720x720/12042845_975095622551586_7459912233832932325_n.jpg?oh=46c36d9469859912aec49fc5fcc41e2d&oe=5689DCA1', u'uploadTime': u'1443647315', u'thumbUrl': u'https://www.facebook.com/opiumbarcelona/photos/pb.143141875746969.-2207520000.1443705403./975095622551586/?type=3&src=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xat1%2Ft31.0-8%2F12080353_975095622551586_7459912233832932325_o.jpg&smallsrc=https%3A%2F%2Fscontent-mad1-1.xx.fbcdn.net%2Fhphotos-xaf1%2Fv%2Ft1.0-9%2F12042845_975095622551586_7459912233832932325_n.jpg%3Foh%3D2685fcc20247d4a0aa155dbbc4ca8485%26oe%3D56988FC3&size=2048%2C1365&fbid=975095622551586', u'albumTitle': u'Mi\xe9rcoles, 30 de septiembre de 2015 a las 14:08', u'fbPageName': u'opiumbarcelona', u'svid': u'5|143141875746969', u'state': 0, u'_id': ObjectId('560d3249aea396329d27bf02')}
{u'urlOfImage': u'https://scontent-mad1-1.xx.fbcdn.net/hphotos-xap1/v/t1.0-9/s720x720/12063698_97509561588

In [148]:
z=collection.find({},  {'state':1, '_id':0})
for i in z:
    print(i)

{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}
{u'state': 0}


In [226]:
#collection1.count()
print(collection1.find_one())

cursor1 = collection1.find()
for record in cursor1[0:10]:
    print(str(record['attribute']['gender']['value']) + ' ' + str(record['attribute']['age']['value']))
    

{u'url': u'https://scontent.xx.fbcdn.net/hphotos-xap1/v/t1.0-9/p720x720/12112147_975868252474323_5519238925163208732_n.png?oh=6a547b4d4cdc6fa5911714c9c1a155ba&oe=568C41FE', u'attribute': {u'gender': {u'confidence': 99.9926, u'value': u'Female'}, u'age': {u'range': 6, u'value': 23}, u'pose': {u'yaw_angle': {u'value': -39.432435}, u'pitch_angle': {u'value': 1.9e-05}, u'roll_angle': {u'value': 10.5119}}, u'smiling': {u'value': 71.8681}, u'race': {u'confidence': 98.8683, u'value': u'White'}}, u'fbPageName': u'opiumbarcelona', u'tag': u'', u'img_width': 720, u'img_height': 720, u'svid': u'5|143141875746969', u'position': {u'eye_left': {u'y': 25.238167, u'x': 24.754833}, u'center': {u'y': 26.666667, u'x': 26.5}, u'width': 6.333333, u'mouth_left': {u'y': 28.512333, u'x': 24.843167}, u'height': 6.333333, u'mouth_right': {u'y': 28.852833, u'x': 27.250667}, u'nose': {u'y': 27.706667, u'x': 25.3725}, u'eye_right': {u'y': 25.7905, u'x': 27.7315}}, u'face_id': u'f931ff0429262fdee5435921fc3d9a96', u